### Preparation Notebook - Exposure

This notebook prepares the exposure datasets by disaggregating sectoral capital stock and GVA data

In [1]:
# Import live code changes in
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os
import geopandas as gpd
import rasterio
import pandas as pd
from functools import reduce

from sovereign.utils import disaggregate_total_to_raster, disaggregate_admin_to_raster

##### 1. Set filepaths

In [2]:
root = Path.cwd().parent.parent # find project root
# Inputs
AGR_path = os.path.join(root, 'inputs', 'exposure', 'gridded', 'THA_cropland.tif')
INF_path = os.path.join(root, 'inputs', 'exposure', 'gridded', 'THA_infra.tif')
RES_path = os.path.join(root, 'inputs', 'exposure', 'gridded', 'THA_ghs-res_v.tif')
NRES_path = os.path.join(root, 'inputs', 'exposure', 'gridded', 'THA_ghs-nres_v.tif')
capstock_path = os.path.join(root, 'inputs', 'exposure', 'capstock', 'giri_capstock.csv')
gva_path = os.path.join(root, 'inputs', 'exposure', 'gva', 'DOSE_V2p11_THA_rgva.gpkg')
# Outputs
inf_pri_path = os.path.join(root, 'outputs', 'exposure', 'inf_priv_capstock.tif')
inf_pub_path = os.path.join(root, 'outputs', 'exposure', 'inf_pub_capstock.tif')
nres_pub_path = os.path.join(root, 'outputs', 'exposure', 'nres_pub_capstock.tif')
nres_pri_path = os.path.join(root, 'outputs', 'exposure', 'nres_priv_capstock.tif')
res_pri_path = os.path.join(root, 'outputs', 'exposure', 'res_priv_capstock.tif')
agr_gva_path = os.path.join(root, 'outputs', 'exposure', 'agr_GVA.tif')
man_gva_path = os.path.join(root, 'outputs', 'exposure', 'man_GVA.tif')
ser_gva_path = os.path.join(root, 'outputs', 'exposure', 'ser_GVA.tif')

##### 2. Disaggregate national capital stock over grid

In [3]:
# Load capstock data
capstock = pd.read_csv(capstock_path)
# Private Infrastructure
inf_pri_sum = capstock.loc[(capstock['variable_type']=='infrastructure') & (capstock['ownership_classification']=='Private'), "data_sum"].sum()
disaggregate_total_to_raster(inf_pri_sum, INF_path, inf_pri_path)
# Public Infrastructure
inf_pub_sum = capstock.loc[(capstock['variable_type']=='infrastructure') & (capstock['ownership_classification']=='Public'), "data_sum"].sum()
disaggregate_total_to_raster(inf_pub_sum, INF_path, inf_pub_path)
# Public Buildings (Non-Residential)
nres_pub_sum = capstock.loc[(capstock['variable_type']=='buildings') & (capstock['ownership_classification']=='Public'), "data_sum"].sum()
disaggregate_total_to_raster(nres_pub_sum, NRES_path, nres_pub_path)
# Private Buildings (Residential)
res_pri_sum = capstock.loc[(capstock['sector_key']=='ic_low') | (capstock['sector_key']=='ic_mlow') | (capstock['sector_key']=='ic_high') | (capstock['sector_key']=='ic_mhigh'), "data_sum"].sum()
disaggregate_total_to_raster(res_pri_sum, RES_path, res_pri_path)
# Private Buildings (Non-Residential)
nres_pri_sum = capstock.loc[(capstock['variable_type']=='buildings') & (capstock['ownership_classification']=='Private'), "data_sum"].sum() - res_pri_sum
disaggregate_total_to_raster(nres_pri_sum, NRES_path, nres_pri_path)

##### 3. Disaggregate sub-national GVA over grid

In [4]:
# Agriculture
disaggregate_admin_to_raster(gva_path, 'ag_grp_usd', AGR_path, agr_gva_path)
# Manufacturing
disaggregate_admin_to_raster(gva_path, 'man_grp_usd', NRES_path, man_gva_path)
# Service
disaggregate_admin_to_raster(gva_path, 'serv_grp_usd', NRES_path, ser_gva_path)